# Telco Customer Churn Prediction - Model Development

This is the third and final notebook in a series of notebooks developing a machine learning model to predict customer churn for a telecommunications company. In the previous notebooks, I cleaned and merged the excel files provided in [IBM's sample datasets](https://accelerator.ca.analytics.ibm.com/bi/?perspective=authoring&pathRef=.public_folders%2FIBM%2BAccelerator%2BCatalog%2FContent%2FDAT00148&id=i9710CF25EF75468D95FFFC7D57D45204&objRef=i9710CF25EF75468D95FFFC7D57D45204&action=run&format=HTML&cmPropStr=%7B%22id%22%3A%22i9710CF25EF75468D95FFFC7D57D45204%22%2C%22type%22%3A%22reportView%22%2C%22defaultName%22%3A%22DAT00148%22%2C%22permissions%22%3A%5B%22execute%22%2C%22read%22%2C%22traverse%22%5D%7D) into a single CSV file containing a consolidated dataset, examined and transformed the dataset features thoroughly and prepared the dataset for the modeling stage.

In this notebook, I will:

- Create preprocessing and modeling pipelines

- Compare baseline models with cross-validation, and alternative class imbalance techniques

- Approach the business problem from three different perspectives, focusing on F1 score, precision and rrecall separately

- Perform light tuning on models to select most promising ones on each metric

- Perform extensive hyperparameter search each model and business objective

- Measure performance of final models on the test set including classification reports, confusion matrices and ROC curve visualizations

- Analyze feature importance and compare results with visualizations from the previous notebook

- Summarize model performance with key drivers of customer churn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_curve, auc
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns', None)

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Loading and inspecting data

In [3]:
telco = pd.read_csv("../data/processed/telco_cleaned.csv", index_col=0)

print(f"Shape: {telco.shape}")
telco.head()

Shape: (7043, 29)


,gender,age,married,number_of_dependents,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_refunds,total_extra_data_charges,churn_value,cltv,streaming,log_total_long_distance_charges,log_total_revenue
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,Female,37,1,0,2,9,no_offer,1,42.39,0,Cable,16,0,1,0,1,1,1,One Year,1,Credit Card,65.6,0.00,0,0,2205,0,5.946755,6.883268
0003-MKNFE,Male,46,0,0,0,9,no_offer,1,10.69,1,Cable,10,0,0,0,0,0,0,Month-to-Month,0,Credit Card,59.9,38.33,10,0,5414,1,4.576874,6.415555
0004-TLHLJ,Male,50,0,0,0,4,Offer E,1,33.65,0,Fiber Optic,30,0,0,1,0,0,1,Month-to-Month,1,Bank Withdrawal,73.9,0.00,0,1,4479,0,4.909709,6.031766
0011-IGKFF,Male,78,1,0,1,13,Offer D,1,27.82,0,Fiber Optic,4,0,1,1,0,1,1,Month-to-Month,1,Bank Withdrawal,98.0,0.00,0,1,3714,0,5.893466,7.378078
0013-EXCHZ,Female,75,1,0,3,3,no_offer,1,7.38,0,Fiber Optic,11,0,0,0,1,1,1,Month-to-Month,1,Credit Card,83.9,0.00,0,1,3464,0,3.141563,5.671741


In [4]:
def cardinality(df, max_display=3):
    res = pd.DataFrame({
        'nunique': df.nunique(),
        'dtype': df.dtypes
    }).sort_values(by='nunique')

    res['unique_values'] = [
        df[col].unique().tolist() if res.loc[col, 'nunique'] <= max_display
        else f"[{', '.join(map(str, df[col].unique()[:max_display]))} ...(cont'd)]"
        for col in res.index
    ]

    res['missing_value_count'] = [
        df[col].isna().sum() for col in df.columns
    ]

    return res

cardinality(telco)

,nunique,dtype,unique_values,missing_value_count
gender,2,object,"[Female, Male]",0
streaming,2,int64,"[0, 1]",0
churn_value,2,int64,"[0, 1]",0
paperless_billing,2,int64,"[1, 0]",0
unlimited_data,2,int64,"[1, 0]",0
streaming_tv,2,int64,"[1, 0]",0
premium_tech_support,2,int64,"[1, 0]",0
online_backup,2,int64,"[1, 0]",0
online_security,2,int64,"[0, 1]",0
device_protection_plan,2,int64,"[0, 1]",0


## 2. Creating a preprocessing pipeline

In [5]:
# Separate target variable
X = telco.drop(['churn_value'], axis=1)
y = telco['churn_value']

# Separate dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

print("\nChurn distribution before splitting:")
print(y.value_counts(normalize=True + 100) * 100)

print("\nTraining set churn distribution:")
print(pd.Series(y_train).value_counts(normalize=True) * 100)

print("\nTest set churn distribution:")
print(pd.Series(y_test).value_counts(normalize=True) * 100)

Training set size: 5634 samples
Test set size: 1409 samples

Churn distribution before splitting:
churn_value
0    73.463013
1    26.536987
Name: proportion, dtype: float64

Training set churn distribution:
churn_value
0    73.464679
1    26.535321
Name: proportion, dtype: float64

Test set churn distribution:
churn_value
0    73.456352
1    26.543648
Name: proportion, dtype: float64


In [6]:
# Identify numerical and categorical features for preprocessing
cat_features = X.select_dtypes(include=['object']).columns.to_list()
num_features = X.select_dtypes(include=['int64', 'float64']).columns.to_list()

print(f"Categorical features ({len(cat_features)}):")
print(cat_features)
print(f"\nNumerical features ({len(num_features)}):")
print(num_features)

Categorical features (5):
['gender', 'offer', 'internet_type', 'contract', 'payment_method']

Numerical features (23):
['age', 'married', 'number_of_dependents', 'number_of_referrals', 'tenure_in_months', 'phone_service', 'avg_monthly_long_distance_charges', 'multiple_lines', 'avg_monthly_gb_download', 'online_security', 'online_backup', 'device_protection_plan', 'premium_tech_support', 'streaming_tv', 'unlimited_data', 'paperless_billing', 'monthly_charge', 'total_refunds', 'total_extra_data_charges', 'cltv', 'streaming', 'log_total_long_distance_charges', 'log_total_revenue']


In [7]:
# Standard scaling for numerical features
num_tr = Pipeline([
    ('scaler', StandardScaler())
])

# One-hot encoding for categorical features
cat_tr = Pipeline([
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
])

# Combine both in a preprocessor
preprocessor = ColumnTransformer([
    ('num', num_tr, num_features),
    ('cat', cat_tr, cat_features)
])

Note that while tree-based models aren't affected by scaling, it doesn't harm them, and maintaining a consistent preprocessing pipeline is cleaner.

## 3. Baseline models and handling class imbalance

In this section, I'll perform an inital evaluation on several baseline models based on their precision, recall and F1 scores via cross-validation. I'll consider two methods to tackle class imbalance: Classs weights and SMOTE. I'll combine the results of these experiments in a dataframe for better comparison.

I'll start by setting up a stratified k-fold which maintains the same class distribution in each fold, and custom scorers which give more flexibility for cross-validation and hyperparameter search processes. Note that `zero_division=0` assigns a precision of 0 when no positive samples are predicted.

In [8]:
# Set up cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Custome scorers fto use within pipelines
f1_scorer = make_scorer(f1_score)
precision_scorer = make_scorer(precision_score, zero_division=0)
recall_scorer = make_scorer(recall_score)

## 3.1 Baseline model fit

In [9]:
models = {
    'Logistic Regression': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    
    'Linear SVM': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LinearSVC(random_state=42, max_iter=2000, dual=False))
    ]),
    
    'Decision Tree': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ]),
    
    'Random Forest': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
    ]),
    
    'XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1))
    ]),
    
    'Gradient Boosting': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(random_state=42))
    ]),
    
    'KNN': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', KNeighborsClassifier(n_jobs=-1))
    ]),
    
    'Naive Bayes': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', GaussianNB())
    ])
}


# Evaluate baseline models
results = {}
for name, model in models.items():
    scores_f1 = cross_val_score(model, X, y, cv=cv, scoring=f1_scorer, n_jobs=-1)
    scores_precision = cross_val_score(model, X, y, cv=cv, scoring=precision_scorer, n_jobs=-1)
    scores_recall = cross_val_score(model, X, y, cv=cv, scoring=recall_scorer, n_jobs=-1)

    results[name] = {
        'mean_f1': scores_f1.mean(),
        'std_f1': scores_f1.std(),
        'mean_precision': scores_precision.mean(),
        'std_precision': scores_precision.std(),
        'mean_recall': scores_recall.mean(),
        'std_recall': scores_recall.std()
    }

In [10]:
# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df

,mean_f1,std_f1,mean_precision,std_precision,mean_recall,std_recall
Logistic Regression,0.691165,0.007773,0.720923,0.020166,0.664514,0.016718
Linear SVM,0.688844,0.007279,0.716662,0.019690,0.663978,0.019136
Decision Tree,0.576750,0.019925,0.576449,0.013452,0.577301,0.027586
Random Forest,0.658425,0.013537,0.737548,0.012407,0.594955,0.019523
XGBoost,0.679385,0.011638,0.712710,0.010947,0.649528,0.021368
Gradient Boosting,0.687245,0.006148,0.741491,0.014704,0.640976,0.017298
KNN,0.596964,0.014018,0.608582,0.010457,0.585886,0.018244
Naive Bayes,0.671586,0.008272,0.581124,0.012289,0.795614,0.002584


## 3.2 Models with balanced class weights

Most models have a `class_weight` parameter. For xgboost I'll calculate `scale_pos_weight` manually. I'll skip gradient boosting, KNN and naive Bayes in this section as they don't have a direct class weight parameter.

In [11]:
balanced_models = {
    'Balanced Logistic Regression': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'))
    ]),
    
    'Balanced Linear SVM': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LinearSVC(random_state=42, max_iter=2000, class_weight='balanced', dual=False))
    ]),
    
    'Balanced Decision Tree': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
    ]),
    
    'Balanced Random Forest': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1))
    ]),
    
    'Balanced XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', xgb.XGBClassifier(
            random_state=42, 
            eval_metric='logloss',
            # Control the balance of positive and negative weights, useful for unbalanced classes. A typical value to consider: sum(negative instances) / sum(positive instances). (source: https://xgboost.readthedocs.io/en/latest/parameter.html)
            scale_pos_weight=(len(y) - sum(y))/sum(y),
            n_jobs=-1
        ))
    ])
}

# Evaluate models with balanced class weights
balanced_results = {}
for name, model in balanced_models.items():
    scores_f1 = cross_val_score(model, X, y, cv=cv, scoring=f1_scorer, n_jobs=-1)
    scores_precision = cross_val_score(model, X, y, cv=cv, scoring=precision_scorer, n_jobs=-1)
    scores_recall = cross_val_score(model, X, y, cv=cv, scoring=recall_scorer, n_jobs=-1)

    balanced_results[name] = {
        'mean_f1': scores_f1.mean(),
        'std_f1': scores_f1.std(),
        'mean_precision': scores_precision.mean(),
        'std_precision': scores_precision.std(),
        'mean_recall': scores_recall.mean(),
        'std_recall': scores_recall.std()
    }

In [12]:
# Convert results to DataFrame for easier comparison
balanced_results_df = pd.DataFrame.from_dict(balanced_results, orient='index')
balanced_results_df

,mean_f1,std_f1,mean_precision,std_precision,mean_recall,std_recall
Balanced Logistic Regression,0.690656,0.007800,0.583714,0.012256,0.845903,0.006096
Balanced Linear SVM,0.684173,0.010312,0.572122,0.014882,0.851251,0.007037
Balanced Decision Tree,0.580355,0.016311,0.587791,0.013076,0.573575,0.025244
Balanced Random Forest,0.656719,0.016702,0.746422,0.006760,0.586925,0.027622
Balanced XGBoost,0.690634,0.005693,0.663127,0.009307,0.721230,0.021707


## 3.3 SMOTE resampling

In order to avoid data leakage, I'll integrate SMOTE resampler into the pipeline containing preprocessing and the model.

In [13]:
smote_models = {
    'SMOTE Logistic Regression': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    
    'SMOTE Linear SVM': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', LinearSVC(random_state=42, max_iter=2000, dual=False))
    ]),
    
    'SMOTE Decision Tree': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ]),
    
    'SMOTE Random Forest': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
    ]),
    
    'SMOTE XGBoost': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1))
    ]),
    
    'SMOTE Gradient Boosting': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', GradientBoostingClassifier(random_state=42))
    ]),

    'SMOTE KNN': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', KNeighborsClassifier(n_jobs=-1))
    ]),
    
    'SMOTE Naive Bayes': ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', GaussianNB())
    ])
}

# Evaluate SMOTE models
smote_results = {}
for name, model in smote_models.items():
    scores_f1 = cross_val_score(model, X, y, cv=cv, scoring=f1_scorer, n_jobs=-1)
    scores_precision = cross_val_score(model, X, y, cv=cv, scoring=precision_scorer, n_jobs=-1)
    scores_recall = cross_val_score(model, X, y, cv=cv, scoring=recall_scorer, n_jobs=-1)

    smote_results[name] = {
        'mean_f1': scores_f1.mean(),
        'std_f1': scores_f1.std(),
        'mean_precision': scores_precision.mean(),
        'std_precision': scores_precision.std(),
        'mean_recall': scores_recall.mean(),
        'std_recall': scores_recall.std()
    }

In [14]:
# Convert results to DataFrame for easier comparison
smote_results_df = pd.DataFrame.from_dict(smote_results, orient='index')
smote_results_df

,mean_f1,std_f1,mean_precision,std_precision,mean_recall,std_recall
SMOTE Logistic Regression,0.692676,0.008247,0.591951,0.014041,0.835197,0.009072
SMOTE Linear SVM,0.686048,0.008418,0.580274,0.014195,0.839479,0.008089
SMOTE Decision Tree,0.598763,0.017827,0.577783,0.013866,0.621701,0.026672
SMOTE Random Forest,0.688720,0.012237,0.698832,0.019194,0.679500,0.018769
SMOTE XGBoost,0.680753,0.010406,0.692836,0.018447,0.669869,0.020889
SMOTE Gradient Boosting,0.706264,0.007746,0.677205,0.014063,0.738354,0.014063
SMOTE KNN,0.590754,0.013432,0.470760,0.010092,0.792938,0.021493
SMOTE Naive Bayes,0.667967,0.007764,0.577174,0.012218,0.792936,0.006037


## 3.4 Combine scores from all experiments

In [15]:
# Combine all results
all_results = pd.concat([results_df, balanced_results_df, smote_results_df])

print("Best models on F1-score:")
display(all_results.sort_values(by='mean_f1', ascending=False)[:5])

print("Best models on precision:")
display(all_results.sort_values(by='mean_precision', ascending=False)[:5])

print("Best models on recall:")
display(all_results.sort_values(by='mean_recall', ascending=False)[:5])

Best models on F1-score:


,mean_f1,std_f1,mean_precision,std_precision,mean_recall,std_recall
SMOTE Gradient Boosting,0.706264,0.007746,0.677205,0.014063,0.738354,0.014063
SMOTE Logistic Regression,0.692676,0.008247,0.591951,0.014041,0.835197,0.009072
Logistic Regression,0.691165,0.007773,0.720923,0.020166,0.664514,0.016718
Balanced Logistic Regression,0.690656,0.007800,0.583714,0.012256,0.845903,0.006096
Balanced XGBoost,0.690634,0.005693,0.663127,0.009307,0.721230,0.021707


Best models on precision:


,mean_f1,std_f1,mean_precision,std_precision,mean_recall,std_recall
Balanced Random Forest,0.656719,0.016702,0.746422,0.006760,0.586925,0.027622
Gradient Boosting,0.687245,0.006148,0.741491,0.014704,0.640976,0.017298
Random Forest,0.658425,0.013537,0.737548,0.012407,0.594955,0.019523
Logistic Regression,0.691165,0.007773,0.720923,0.020166,0.664514,0.016718
Linear SVM,0.688844,0.007279,0.716662,0.019690,0.663978,0.019136


Best models on recall:


,mean_f1,std_f1,mean_precision,std_precision,mean_recall,std_recall
Balanced Linear SVM,0.684173,0.010312,0.572122,0.014882,0.851251,0.007037
Balanced Logistic Regression,0.690656,0.007800,0.583714,0.012256,0.845903,0.006096
SMOTE Linear SVM,0.686048,0.008418,0.580274,0.014195,0.839479,0.008089
SMOTE Logistic Regression,0.692676,0.008247,0.591951,0.014041,0.835197,0.009072
Naive Bayes,0.671586,0.008272,0.581124,0.012289,0.795614,0.002584


## 4. Modeling strategy

This project's ultimate goal is to build prediction models for customer churn and interpret which features are important in this process. The business problem can be approached from several perspectives:

- **Recall-focused approach**: Prioritizing catching as many potential churners as possible, minimizing false negatives. This is valuable when the cost of missing a churner is high.

- **Precision-focused approach**: Prioritizing identifying true churners with fewer false alarms, minimizing false positives. This is valuable when intervention resources are limited.

- **Balanced approach**: Optimizing for overall model performance by balancing false positives and false negatives, usually measured by F1-score.

### Approach

I'll implement a two-stage hyperparameter tuning process:

1. **Initial Light Tuning**: Perform a lightweight hyperparameter search on all models to get a better sense of their potential.

2. **Final Deep Tuning**: Select the most promising models for each business objective based on the initial tuning results, then perform a more extensive hyperparameter search.

For class imbalance handling, I'll use class weights since:
- The target imbalance is not severe (approximately 27% to 73%)
- The experiment results don't show SMOTE providing significantly better performance
- Class weights preserve the original data distribution while addressing the imbalance

## 5. Initial light hyperparameter tuning

I'll perform a lightweight hyperparameter search on all models before selecting the best performers for each metric.

In [16]:
# Helper function for light hyperparameter tuning
def light_tune_model(model_class, param_grid, scoring, model_name):
    """
    Performs lightweight hyperparameter tuning for a model
    """
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model_class)
    ])
    
    grid_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_grid,
        n_iter=5,  # Reduced iterations for lightweight search
        scoring=scoring,
        cv=cv,
        random_state=42,
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train, y_train)
    
    return {
        'model_name': model_name,
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_,
        'best_estimator': grid_search.best_estimator_
    }

In [17]:
# Lightweight parameter grids for each model type
lr_light_params = {
    'model__C': [0.1, 1, 10],
    'model__penalty': ['l1', 'l2'],
    'model__class_weight': ['balanced', None]
}

svm_light_params = {
    'model__C': [0.1, 1, 10],
    'model__class_weight': ['balanced', None],
    'model__dual': [False]
}

tree_light_params = {
    'model__max_depth': [5, 10, None],
    'model__min_samples_split': [2, 5],
    'model__class_weight': ['balanced', None]
}

rf_light_params = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [5, 10, None],
    'model__class_weight': ['balanced', None]
}

xgb_light_params = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [3, 5],
    'model__learning_rate': [0.01, 0.1],
    'model__scale_pos_weight': [1, (len(y) - sum(y))/sum(y)]
}

gb_light_params = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [3, 5],
    'model__learning_rate': [0.01, 0.1]
}

knn_light_params = {
    'model__n_neighbors': [3, 5, 7],
    'model__weights': ['uniform', 'distance']
}

nb_light_params = {
    'model__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}


## 5.1 Perform light tuning for F1 score optimization

In [18]:
f1_tuning_results = []

f1_tuning_results.append(light_tune_model(
    LogisticRegression(random_state=42, solver='saga', max_iter=2000), 
    lr_light_params, f1_scorer, 'Logistic Regression'
))

f1_tuning_results.append(light_tune_model(
    LinearSVC(random_state=42, max_iter=2000), 
    svm_light_params, f1_scorer, 'Linear SVM'
))

f1_tuning_results.append(light_tune_model(
    DecisionTreeClassifier(random_state=42), 
    tree_light_params, f1_scorer, 'Decision Tree'
))

f1_tuning_results.append(light_tune_model(
    RandomForestClassifier(random_state=42, n_jobs=-1), 
    rf_light_params, f1_scorer, 'Random Forest'
))

f1_tuning_results.append(light_tune_model(
    xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1), 
    xgb_light_params, f1_scorer, 'XGBoost'
))

f1_tuning_results.append(light_tune_model(
    GradientBoostingClassifier(random_state=42), 
    gb_light_params, f1_scorer, 'Gradient Boosting'
))

f1_tuning_results.append(light_tune_model(
    KNeighborsClassifier(n_jobs=-1), 
    knn_light_params, f1_scorer, 'KNN'
))

f1_tuning_results.append(light_tune_model(
    GaussianNB(), 
    nb_light_params, f1_scorer, 'Naive Bayes'
))

# Sort results by F1 score
f1_tuning_results.sort(key=lambda x: x['best_score'], reverse=True)

# Display top models
print("Top models for F1 score after light tuning:")
for i, result in enumerate(f1_tuning_results[:3]):
    print(f"\n{i+1}. {result['model_name']}: {result['best_score']:.4f}")
    print(f"Parameters: {result['best_params']}")

Top models for F1 score after light tuning:

1. XGBoost: 0.7073
Parameters: {'model__scale_pos_weight': 2.7683253076511503, 'model__n_estimators': 50, 'model__max_depth': 5, 'model__learning_rate': 0.1}

2. Linear SVM: 0.6987
Parameters: {'model__dual': False, 'model__class_weight': None, 'model__C': 10}

3. Logistic Regression: 0.6984
Parameters: {'model__penalty': 'l1', 'model__class_weight': None, 'model__C': 10}


## 5.2 Perform light tuning for precision optimization

In [19]:
precision_tuning_results = []

precision_tuning_results.append(light_tune_model(
    LogisticRegression(random_state=42, solver='saga', max_iter=2000), 
    lr_light_params, precision_scorer, 'Logistic Regression'
))

precision_tuning_results.append(light_tune_model(
    LinearSVC(random_state=42, max_iter=2000), 
    svm_light_params, precision_scorer, 'Linear SVM'
))

precision_tuning_results.append(light_tune_model(
    DecisionTreeClassifier(random_state=42), 
    tree_light_params, precision_scorer, 'Decision Tree'
))

precision_tuning_results.append(light_tune_model(
    RandomForestClassifier(random_state=42, n_jobs=-1), 
    rf_light_params, precision_scorer, 'Random Forest'
))

precision_tuning_results.append(light_tune_model(
    xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1), 
    xgb_light_params, precision_scorer, 'XGBoost'
))

precision_tuning_results.append(light_tune_model(
    GradientBoostingClassifier(random_state=42), 
    gb_light_params, precision_scorer, 'Gradient Boosting'
))

precision_tuning_results.append(light_tune_model(
    KNeighborsClassifier(n_jobs=-1), 
    knn_light_params, precision_scorer, 'KNN'
))

precision_tuning_results.append(light_tune_model(
    GaussianNB(), 
    nb_light_params, precision_scorer, 'Naive Bayes'
))

# Sort results by precision
precision_tuning_results.sort(key=lambda x: x['best_score'], reverse=True)

# Display top models
print("Top models for Precision after light tuning:")
for i, result in enumerate(precision_tuning_results[:3]):
    print(f"\n{i+1}. {result['model_name']}: {result['best_score']:.4f}")
    print(f"Parameters: {result['best_params']}")

Top models for Precision after light tuning:

1. Gradient Boosting: 0.8915
Parameters: {'model__n_estimators': 100, 'model__max_depth': 3, 'model__learning_rate': 0.01}

2. Random Forest: 0.7528
Parameters: {'model__n_estimators': 100, 'model__max_depth': None, 'model__class_weight': 'balanced'}

3. XGBoost: 0.7357
Parameters: {'model__scale_pos_weight': 1, 'model__n_estimators': 100, 'model__max_depth': 5, 'model__learning_rate': 0.1}


## 5.3 Perform light tuning for recall optimization

In [20]:
recall_tuning_results = []

recall_tuning_results.append(light_tune_model(
    LogisticRegression(random_state=42, solver='saga', max_iter=2000), 
    lr_light_params, recall_scorer, 'Logistic Regression'
))

recall_tuning_results.append(light_tune_model(
    LinearSVC(random_state=42, max_iter=2000), 
    svm_light_params, recall_scorer, 'Linear SVM'
))

recall_tuning_results.append(light_tune_model(
    DecisionTreeClassifier(random_state=42), 
    tree_light_params, recall_scorer, 'Decision Tree'
))

recall_tuning_results.append(light_tune_model(
    RandomForestClassifier(random_state=42, n_jobs=-1), 
    rf_light_params, recall_scorer, 'Random Forest'
))

recall_tuning_results.append(light_tune_model(
    xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1), 
    xgb_light_params, recall_scorer, 'XGBoost'
))

recall_tuning_results.append(light_tune_model(
    GradientBoostingClassifier(random_state=42), 
    gb_light_params, recall_scorer, 'Gradient Boosting'
))

recall_tuning_results.append(light_tune_model(
    KNeighborsClassifier(n_jobs=-1), 
    knn_light_params, recall_scorer, 'KNN'
))

recall_tuning_results.append(light_tune_model(
    GaussianNB(), 
    nb_light_params, recall_scorer, 'Naive Bayes'
))

# Sort results by recall
recall_tuning_results.sort(key=lambda x: x['best_score'], reverse=True)

# Display top models
print("Top models for Recall after light tuning:")
for i, result in enumerate(recall_tuning_results[:3]):
    print(f"\n{i+1}. {result['model_name']}: {result['best_score']:.4f}")
    print(f"Parameters: {result['best_params']}")

Top models for Recall after light tuning:

1. XGBoost: 0.8870
Parameters: {'model__scale_pos_weight': 2.7683253076511503, 'model__n_estimators': 50, 'model__max_depth': 5, 'model__learning_rate': 0.01}

2. Linear SVM: 0.8589
Parameters: {'model__dual': False, 'model__class_weight': 'balanced', 'model__C': 1}

3. Logistic Regression: 0.8508
Parameters: {'model__penalty': 'l1', 'model__class_weight': 'balanced', 'model__C': 0.1}
